In [ ]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test
coll = db.restaurants

# B-tree indexes

In [ ]:
coll.drop_indexes()

In [ ]:
coll.index_information()

In [ ]:
coll.find_one()

In [ ]:
coll.create_index('name')

In [ ]:
coll.create_index([('name', 1)])

In [ ]:
coll.create_index([('name', pymongo.ASCENDING)])

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
q.count()

In [ ]:
set(doc['cuisine'] for doc in q)

In [ ]:
coll.create_index([('name', 1), ('cuisine', 1)])

In [ ]:
coll.drop_index('name_1')

In [ ]:
q = coll.find(
    {'name': 'Dunkin Donuts', 'cuisine': 'Armenian'},
    {'name': 1, 'borough': 1, '_id': 0})
list(q)    

# Using `explain()`

In [ ]:
coll.drop_indexes()

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan

In [ ]:
plan['queryPlanner']['winningPlan']

In [ ]:
plan['executionStats']['executionStages']

In [ ]:
from pprint import pprint
def plan_summary(plan):
    winningPlan = plan['queryPlanner']['winningPlan']
    executionStages = plan['executionStats']['executionStages']
    print('Winning Plan')
    pprint(winningPlan)
    print()
    print('Execution Stages')
    pprint(executionStages)

In [ ]:
plan_summary(plan)

In [ ]:
coll.create_index('name')

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan_summary(plan)

In [ ]:
plan['executionStats']['executionStages']

In [ ]:
from pprint import pprint
def plan_summary(plan):
    winningPlan = plan['queryPlanner']['winningPlan']
    stages = []
    stage = plan['executionStats']['executionStages']
    while stage:
        inputStage = stage.pop('inputStage', None)
        stages.append(stage)
        stage = inputStage
    stages.reverse()
    print('Winning Plan')
    pprint(winningPlan)
    print()
    print('Execution Stages')
    for stage in stages:
        pprint(stage)
        print('--')

In [ ]:
plan_summary(plan)

In [ ]:
q = coll.find({'name': 'Dunkin Donuts', 'cuisine': 'Armenian'})
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.create_index([('name', 1), ('cuisine', 1)])
coll.drop_index('name_1')

In [ ]:
q = coll.find({'name': 'Dunkin Donuts', 'cuisine': 'Armenian'})
plan = q.explain()
plan_summary(plan)

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan_summary(plan)

### Range / prefix queries

In [ ]:
import re
q = coll.find({'name': re.compile('^Dunkin.* Donuts$')})
plan = q.explain()
plan_summary(plan)

In [ ]:
q.distinct('name')

In [ ]:
coll.drop_indexes()
coll.create_index([('cuisine', 1), ('name', 1)])

In [ ]:
q = coll.find({'name': re.compile('^Dunkin.* Donuts$')})
plan = q.explain()
plan_summary(plan)

In [ ]:
q = coll.find({'name': re.compile('^Dunkin.* Donuts$'), 'cuisine': 'Donuts'})
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.find_one()

In [ ]:
coll.drop_indexes()
coll.create_index('name')

In [ ]:
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.create_index([('name',1), ('address.zipcode', 1)])

In [ ]:
q = coll.find({'name': 'Dunkin Donuts', 'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
plan = q.explain()
plan_summary(plan)

# Sorting

In [ ]:
coll.drop_indexes()
coll.create_index('name')
q = coll.find({'name': 'Dunkin Donuts'})
q = q.sort('address.zipcode')
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.drop_indexes()
coll.create_index([('name', 1), ('address.zipcode', 1)])

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
q = q.sort('address.zipcode')
plan = q.explain()
plan_summary(plan)

In [ ]:
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
q = q.sort('borough')
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.drop_indexes()
coll.create_index([
    ('name', 1),
    ('borough', 1),
    ('address.zipcode', 1)
])
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
q = q.sort('borough')
plan = q.explain()
plan_summary(plan)

## Sorting with Ranges Trade-off

- Sort before range examines more _index keys_
- Range before sort generates more _stages_ in the query

Always arrange you index with

1. Exact matches
1. Sort keys
1. Range matches

... in that order